# 🛒 Kaspi.kz Cards Parser

Парсер **карточек товаров** со всех категорий Kaspi.kz с записью в ClickHouse.

## ⚡ Особенности:
- **50 параллельных потоков** (asyncio + aiohttp)
- **Retry с exponential backoff** (до 3 попыток)
- **Пагинация** — собирает все страницы каждой категории
- **Статистика** — RPS, ретраи, ошибки
- **ClickHouse** — автоматическая запись в таблицу `cards`

---
## 1️⃣ Импорты и настройки

In [ ]:
import pandas as pd
import requests
import aiohttp
import asyncio
import json
import time
import urllib3
from datetime import datetime
from dataclasses import dataclass, field
from typing import List, Dict, Any, Optional
from collections import defaultdict

# Отключаем предупреждения SSL
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Загружаем конфиг
with open('config.json', 'r') as f:
    CONFIG = json.load(f)

CH = CONFIG['clickhouse']

print('✅ Настройки загружены')
print(f'📊 ClickHouse: {CH["CH_HOST"]}:{CH["CH_PORT"]}/{CH["CH_DB"]}.cards')

---
## 2️⃣ Конфигурация парсера

In [ ]:
# ============== НАСТРОЙКИ ПАРСЕРА ==============
MAX_CONCURRENT = 50          # Параллельных запросов
MAX_RETRIES = 3              # Максимум ретраев
RETRY_DELAY = 1.0            # Начальная задержка для retry (секунды)
REQUEST_TIMEOUT = 15         # Таймаут запроса (секунды)
ITEMS_PER_PAGE = 24          # Товаров на странице (Kaspi default)
MAX_PAGES_PER_CATEGORY = 100 # Лимит страниц на категорию (для теста)
# ===============================================

# Заголовки для Kaspi API
HEADERS = {
    'Accept': 'application/json, text/*',
    'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8,ru;q=0.7',
    'Connection': 'keep-alive',
    'Cookie': 'ks.tg=35; kaspi.storefront.cookie.city=750000000',
    'Referer': 'https://kaspi.kz/shop/c/categories/',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36',
    'X-KS-City': '750000000',
}

print(f'⚙️ Параллельных потоков: {MAX_CONCURRENT}')
print(f'🔄 Максимум ретраев: {MAX_RETRIES}')
print(f'⏱️ Таймаут запроса: {REQUEST_TIMEOUT}s')

---
## 3️⃣ Класс статистики

In [ ]:
@dataclass
class Stats:
    """Статистика парсинга"""
    start_time: float = 0
    end_time: float = 0
    total_requests: int = 0
    successful_requests: int = 0
    failed_requests: int = 0
    total_retries: int = 0
    total_cards: int = 0
    categories_processed: int = 0
    errors: List[Dict] = field(default_factory=list)
    retries_by_attempt: Dict[int, int] = field(default_factory=lambda: defaultdict(int))
    
    def start(self):
        self.start_time = time.time()
    
    def stop(self):
        self.end_time = time.time()
    
    @property
    def duration(self) -> float:
        return self.end_time - self.start_time
    
    @property
    def rps(self) -> float:
        if self.duration > 0:
            return self.total_requests / self.duration
        return 0
    
    @property
    def success_rate(self) -> float:
        if self.total_requests > 0:
            return (self.successful_requests / self.total_requests) * 100
        return 0
    
    def report(self) -> str:
        return f"""
╔══════════════════════════════════════════════════════════════╗
║                    📊 СТАТИСТИКА ПАРСИНГА                    ║
╠══════════════════════════════════════════════════════════════╣
║  ⏱️  Время работы:          {self.duration:>8.2f} сек                   ║
║  📦  Собрано карточек:      {self.total_cards:>8,}                       ║
║  📁  Категорий обработано:  {self.categories_processed:>8,}                       ║
╠══════════════════════════════════════════════════════════════╣
║  🌐  Всего запросов:        {self.total_requests:>8,}                       ║
║  ✅  Успешных:              {self.successful_requests:>8,}                       ║
║  ❌  Неудачных:             {self.failed_requests:>8,}                       ║
║  📈  Success rate:          {self.success_rate:>8.1f}%                      ║
╠══════════════════════════════════════════════════════════════╣
║  🚀  Запросов в секунду:    {self.rps:>8.2f} RPS                    ║
║  🔄  Всего ретраев:         {self.total_retries:>8,}                       ║
╚══════════════════════════════════════════════════════════════╝
"""

# Глобальная статистика
stats = Stats()
print('✅ Класс статистики готов')

---
## 4️⃣ Функция записи в ClickHouse

In [ ]:
def insert_df(df: pd.DataFrame, table: str, database: str = None) -> int:
    """Записывает DataFrame в ClickHouse через HTTP интерфейс"""
    if df.empty:
        return 0
        
    db = database or CH['CH_DB']
    url = f"http://{CH['CH_HOST']}:{CH['CH_PORT']}/"
    
    # Конвертируем DataFrame в JSONEachRow формат
    records = df.to_dict(orient='records')
    data = '\n'.join(json.dumps(r, ensure_ascii=False, default=str) for r in records)
    
    # POST запрос
    query = f"INSERT INTO {table} FORMAT JSONEachRow"
    
    r = requests.post(
        url,
        params={"database": db, "query": query},
        data=data.encode('utf-8'),
        auth=(CH['CH_USER'], CH['CH_PASS']),
        timeout=CH['CH_HTTP_TIMEOUT']
    )
    
    if r.status_code != 200:
        raise Exception(f"ClickHouse error: {r.text}")
    
    return len(records)

print('✅ Функция insert_df готова')

---
## 5️⃣ Получение списка категорий

In [ ]:
def get_all_categories() -> List[Dict]:
    """Получает все категории для парсинга"""
    url = 'https://kaspi.kz/yml/product-view/pl/filters'
    
    # Сначала получаем основные категории
    main_url = 'https://kaspi.kz/yml/main-navigation/n/n/desktop-menu'
    params = {'depth': '1', 'city': '750000000', 'rootType': 'desktop'}
    
    response = requests.get(main_url, params=params, headers=HEADERS, verify=False, timeout=10)
    main_data = response.json()
    
    all_categories = []
    
    for node in main_data['subNodes']:
        code = node['code']
        title = node['title']
        
        # Получаем подкатегории
        params = {
            'q': f':category:{code}:availableInZones:Magnum_ZONE1',
            'text': '', 'all': 'false', 'sort': 'relevance',
            'ui': 'd', 'i': '-1', 'c': '750000000'
        }
        
        try:
            resp = requests.get(url, params=params, headers=HEADERS, verify=False, timeout=10)
            data = resp.json()
            tree = data.get('data', {}).get('treeCategory', {})
            
            # Собираем leaf категории (без подкатегорий) для парсинга
            def extract_leaves(items, parent_code, parent_title):
                leaves = []
                for item in items:
                    cat_id = item['id'].replace(':category:', '')
                    if 'items' in item and item['items']:
                        leaves.extend(extract_leaves(item['items'], cat_id, item['title']))
                    else:
                        leaves.append({
                            'code': cat_id,
                            'title': item['title'],
                            'count': item.get('count', 0),
                            'main_category': code,
                            'main_title': title
                        })
                return leaves
            
            if 'items' in tree and tree['items']:
                leaves = extract_leaves(tree['items'], code, title)
                all_categories.extend(leaves)
                
        except Exception as e:
            print(f'⚠️ Ошибка получения категорий {code}: {e}')
        
        time.sleep(0.3)
    
    return all_categories

print('Получаем список категорий...')
categories = get_all_categories()
print(f'✅ Найдено {len(categories)} leaf-категорий для парсинга')

# Показываем первые 10
pd.DataFrame(categories[:10])

---
## 6️⃣ Асинхронный парсер карточек

In [ ]:
async def fetch_page(session: aiohttp.ClientSession, category: Dict, page: int, semaphore: asyncio.Semaphore) -> List[Dict]:
    """Получает одну страницу товаров с ретраями"""
    url = 'https://kaspi.kz/yml/product-view/pl/results'
    
    params = {
        'q': f':category:{category["code"]}:availableInZones:Magnum_ZONE1',
        'text': '',
        'sort': 'relevance',
        'ui': 'd',
        'i': str(page * ITEMS_PER_PAGE),  # offset
        'c': '750000000'
    }
    
    async with semaphore:
        for attempt in range(MAX_RETRIES):
            try:
                stats.total_requests += 1
                
                async with session.get(url, params=params, timeout=aiohttp.ClientTimeout(total=REQUEST_TIMEOUT)) as response:
                    if response.status == 200:
                        data = await response.json()
                        cards = data.get('data', {}).get('cards', [])
                        stats.successful_requests += 1
                        return cards
                    else:
                        raise Exception(f'HTTP {response.status}')
                        
            except Exception as e:
                stats.total_retries += 1
                stats.retries_by_attempt[attempt + 1] += 1
                
                if attempt < MAX_RETRIES - 1:
                    delay = RETRY_DELAY * (2 ** attempt)  # Exponential backoff
                    await asyncio.sleep(delay)
                else:
                    stats.failed_requests += 1
                    stats.errors.append({
                        'category': category['code'],
                        'page': page,
                        'error': str(e)
                    })
                    return []
    
    return []


async def parse_category(session: aiohttp.ClientSession, category: Dict, semaphore: asyncio.Semaphore) -> List[Dict]:
    """Парсит все страницы одной категории"""
    all_cards = []
    total_items = category.get('count', 0)
    total_pages = min((total_items // ITEMS_PER_PAGE) + 1, MAX_PAGES_PER_CATEGORY)
    
    # Параллельно запрашиваем все страницы категории
    tasks = [fetch_page(session, category, page, semaphore) for page in range(total_pages)]
    results = await asyncio.gather(*tasks)
    
    for cards in results:
        for card in cards:
            card['category_code'] = category['code']
            card['category_title'] = category['title']
            card['main_category'] = category['main_category']
            card['main_title'] = category['main_title']
            card['parsed_at'] = datetime.now().isoformat()
        all_cards.extend(cards)
    
    return all_cards


async def parse_all_categories(categories: List[Dict]) -> List[Dict]:
    """Главная функция парсинга"""
    all_cards = []
    semaphore = asyncio.Semaphore(MAX_CONCURRENT)
    
    connector = aiohttp.TCPConnector(limit=MAX_CONCURRENT, ssl=False)
    
    async with aiohttp.ClientSession(headers=HEADERS, connector=connector) as session:
        # Обрабатываем категории батчами для отображения прогресса
        batch_size = 10
        
        for i in range(0, len(categories), batch_size):
            batch = categories[i:i+batch_size]
            
            tasks = [parse_category(session, cat, semaphore) for cat in batch]
            results = await asyncio.gather(*tasks)
            
            for cards in results:
                all_cards.extend(cards)
                stats.categories_processed += 1
            
            # Прогресс
            progress = min(i + batch_size, len(categories))
            print(f'\r📦 Обработано: {progress}/{len(categories)} категорий, карточек: {len(all_cards):,}', end='')
    
    print()  # Новая строка
    return all_cards

print('✅ Парсер готов')

---
## 7️⃣ Запуск парсинга

⚠️ **Внимание:** Для теста можно ограничить количество категорий

In [ ]:
# Для теста берём первые N категорий (закомментируй для полного парсинга)
TEST_LIMIT = 20  # None для всех категорий

categories_to_parse = categories[:TEST_LIMIT] if TEST_LIMIT else categories
print(f'🚀 Запускаем парсинг {len(categories_to_parse)} категорий...')
print(f'⚡ Параллельных потоков: {MAX_CONCURRENT}')
print('-' * 60)

# Сброс статистики
stats = Stats()
stats.start()

# Запуск парсинга
all_cards = await parse_all_categories(categories_to_parse)

stats.stop()
stats.total_cards = len(all_cards)

print('-' * 60)
print(f'✅ Парсинг завершён!')

---
## 8️⃣ Статистика

In [ ]:
print(stats.report())

# Детали по ретраям
if stats.retries_by_attempt:
    print('\n📊 Ретраи по попыткам:')
    for attempt, count in sorted(stats.retries_by_attempt.items()):
        print(f'   Попытка {attempt}: {count} ретраев')

# Ошибки
if stats.errors:
    print(f'\n❌ Первые 5 ошибок:')
    for err in stats.errors[:5]:
        print(f"   {err['category']} page {err['page']}: {err['error']}")

---
## 9️⃣ Подготовка DataFrame

In [ ]:
if all_cards:
    df = pd.DataFrame(all_cards)
    
    # Выбираем и переименовываем нужные колонки
    columns_map = {
        'id': 'product_id',
        'title': 'title',
        'brand': 'brand',
        'unitPrice': 'price',
        'reviewsQuantity': 'reviews_count',
        'rating': 'rating',
        'previewImages': 'images',
        'category_code': 'category_code',
        'category_title': 'category_title',
        'main_category': 'main_category',
        'main_title': 'main_title',
        'parsed_at': 'parsed_at'
    }
    
    # Оставляем только существующие колонки
    available_cols = [c for c in columns_map.keys() if c in df.columns]
    df_clean = df[available_cols].rename(columns={k: columns_map[k] for k in available_cols})
    
    # Конвертируем типы
    if 'price' in df_clean.columns:
        df_clean['price'] = pd.to_numeric(df_clean['price'], errors='coerce').fillna(0).astype(int)
    if 'reviews_count' in df_clean.columns:
        df_clean['reviews_count'] = pd.to_numeric(df_clean['reviews_count'], errors='coerce').fillna(0).astype(int)
    if 'rating' in df_clean.columns:
        df_clean['rating'] = pd.to_numeric(df_clean['rating'], errors='coerce').fillna(0)
    if 'images' in df_clean.columns:
        df_clean['images'] = df_clean['images'].apply(lambda x: json.dumps(x) if isinstance(x, list) else '[]')
    
    # Заполняем пустые строки
    str_cols = ['product_id', 'title', 'brand', 'category_code', 'category_title', 'main_category', 'main_title']
    for col in str_cols:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].fillna('')
    
    print(f'✅ DataFrame готов: {len(df_clean):,} строк, {len(df_clean.columns)} колонок')
    print(f'\nКолонки: {list(df_clean.columns)}')
    df_clean.head(10)
else:
    print('❌ Нет данных для обработки')
    df_clean = pd.DataFrame()

---
## 🔟 Запись в ClickHouse

In [ ]:
if not df_clean.empty:
    table = 'cards'
    db = CH['CH_DB']
    
    print(f'📤 Записываем {len(df_clean):,} строк в {db}.{table}...')
    
    try:
        start = time.time()
        inserted = insert_df(df_clean, table, db)
        elapsed = time.time() - start
        
        print(f'✅ Успешно записано {inserted:,} строк за {elapsed:.2f} сек')
        print(f'📊 Скорость записи: {inserted/elapsed:,.0f} строк/сек')
    except Exception as e:
        print(f'❌ Ошибка записи: {e}')
else:
    print('⚠️ Нечего записывать')

---
## 📋 Просмотр данных

In [ ]:
if not df_clean.empty:
    display(df_clean)
else:
    print('Нет данных')